In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [21]:
location = '../maindata/'
events = pd.DataFrame.from_csv(location+"user_activity.csv",index_col=None)
structure = pd.DataFrame.from_csv(location+"structure.csv",index_col=None)
targets = pd.DataFrame.from_csv(location+"targets.csv",index_col=None)
events_test = pd.DataFrame.from_csv(location+"user_activity_test.csv",index_col=None)

In [22]:
def getX(ev):
    'Функция формирует входные вектора для алгоритма RandomForest'
    sc  = ev.groupby('user_id')
    return (sc.sum().index, sc.sum().step_cost.values.reshape((-1, 1)))

In [23]:
ind, X = getX(events)

In [24]:
ind_test, X_test = getX(events_test)

In [25]:
Y = targets.passed.values

In [51]:
#model = DummyClassifier( constant=0)
model =  RandomForestClassifier()

In [60]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
#Поделим нашу выборку на train и validation
Xtr,Xval,Ytr,Yval = train_test_split(X,Y,test_size=0.1,random_state=128)

In [61]:
#Обучим модель
model.fit(Xtr, Ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [62]:
#Оценим метрику на validation
print(f1_score(Yval, model.predict(Xval)))

0.22784810126582278

In [ ]:
def create_submission(X, name):
    np.savetxt('%s.csv'%name, X, delimiter=',', fmt="%d", header='user_id,passed', comments='')

In [63]:
#Предскажем результат и запишем его в .csv
ans = model.predict(X_test)
result = np.concatenate((np.asarray(ind_test, dtype=int).reshape(-1, 1), np.asarray(ans, dtype=int).reshape(-1, 1)), axis=1)
create_submission(result, 'submissions/random_forrest')